In [1]:


!pip install opencv-python numpy pillow sklearn tensorflow mtcnn pandas

  Using cached opencv_python-4.6.0.66-cp36-abi3-win_amd64.whl (35.6 MB)
  Using cached Pillow-8.4.0-cp36-cp36m-win_amd64.whl (3.2 MB)
  Using cached sklearn-0.0.tar.gz (1.1 kB)
  Using cached mtcnn-0.1.1-py3-none-any.whl (2.3 MB)
  Using cached Keras_Preprocessing-1.1.2-py2.py3-none-any.whl (42 kB)
  Using cached termcolor-1.1.0.tar.gz (3.9 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Using cached gast-0.4.0-py3-none-any.whl (9.8 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached flatbuffers-1.12-py2.py3-none-any.whl (15 kB)
  Using cached pytz-2022.1-py2.py3-none-any.whl (503 kB)
  Using cached tensorboard_plugin_wit-1.8.1-py3-none-any.whl (781 kB)
  Using cached tensorboard_data_server-0.6.1-py3-none-any.whl (2.4 kB)
  Using cached google_auth_oauthlib-0.4.6-py2.py3-none-any.whl (18 kB)
  Using cached requests_oauthlib-1.3.1-py2.py3-none-any.whl (23 kB)
  Using cached importlib_m

In [1]:
import cv2
import os
import joblib
import numpy as np
import time
from PIL import Image
from sklearn.svm import SVC
from mtcnn.mtcnn import MTCNN
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import load_model
from sklearn.preprocessing import Normalizer, LabelEncoder

In [2]:

try:os.makedirs('faces')
except:pass

try:os.makedirs('faces/train')
except:pass

try:os.makedirs('faces/val')
except:pass



name = input('Enter your name --> ')
roll = input('Enter your roll no --> ')

name = name+'-'+roll



if name in os.listdir('faces/train'):
    print('User already exist in faces')

else:    
    os.makedirs('faces/train/'+name)
    os.makedirs('faces/val/'+name)

    cap = cv2.VideoCapture(0)
    i = 0
    print()
    for i in range(5):
        print(f'Capturing starts in {5-i} seconds...')
        time.sleep(1)
    print('Taking photos...')
    while i<=200:
        ret,frame = cap.read()
        cv2.imshow('taking your pictures',frame)
        if i%5==0 and i<=150 and i!=0:
            cv2.imwrite('faces/train/'+name+'/'+str(i)+'.png',frame)
        elif i%5==0 and i>150:
            cv2.imwrite('faces/val/'+name+'/'+str(i)+'.png',frame)
        i+=1

    cv2.destroyAllWindows()
    cap.release()
    print('Successfully taken your photos...')




embedding_model = load_model('models/facenet_keras.h5')
print('Embedding Model Loaded')


detector = MTCNN()

def find_face(path,img_size=(160,160)):
    img = cv2.imread(path)
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    img = np.asarray(img) # converting our image obj to numpy array
    faces = detector.detect_faces(img)
    if faces:
        x,y,w,h = faces[0]['box']
        x,y=abs(x),abs(y)
        face = img[y:y+h,x:x+w]
        face = Image.fromarray(face) # converting it to image object to resize it
        face = face.resize(img_size) # resizing it
        face = np.asarray(face)      # converting it back to array
        return face
    return None


def embed(face):
    face = face.astype('float32')
    fm,fs = face.mean(),face.std()
    face = (face-fm)/fs # standardizing the data 
    face = np.expand_dims(face,axis=0) # flattening it
    embs = embedding_model.predict(face) # embedding model converts our160*160*3 vector to 128 features
    return embs[0]



def load_dataset(path):
    X = []
    y = []
    for people in os.listdir(path):
        for people_images in os.listdir(path+people):
            face = find_face(path+people+'/'+people_images)
            if face is None:continue
            emb = embed(face)
            X.append(emb)
            y.append(people)
        print('Loaded {} images of {}'.format(len(os.listdir(path+'/'+people)),people)) 
    return np.asarray(X),np.asarray(y)


print('Loading train data...')
X_train, y_train = load_dataset('faces/train/')

print()

print('Loading test data...')
X_test, y_test = load_dataset('faces/val/')


l2_normalizer = Normalizer('l2')

X_train = l2_normalizer.transform(X_train)
X_test  = l2_normalizer.transform(X_test)


label_enc = LabelEncoder()
y_train = label_enc.fit_transform(y_train)
y_test = label_enc.transform(y_test)


svc = SVC(kernel='linear',probability=True)
svc.fit(X_train,y_train)
joblib.dump(svc,'models/face_prediction_model.sav')
print()

print('SVM Model saved successfully!!')

Enter your name --> leela prasad sir
Enter your roll no --> 555

Capturing starts in 5 seconds...
Capturing starts in 4 seconds...
Capturing starts in 3 seconds...
Capturing starts in 2 seconds...
Capturing starts in 1 seconds...
Taking photos...
Successfully taken your photos...
Embedding Model Loaded
Loading train data...
Loaded 30 images of brock-1234
Loaded 30 images of leela prasad sir-555

Loading test data...
Loaded 10 images of brock-1234
Loaded 10 images of leela prasad sir-555

SVM Model saved successfully!!
